# 🦾 MILESTONE 1: AVENGERS GUARD ACTIVATION
## Voice Command Detection System

**Objective**: Activate the AI guard agent using voice commands like:
- "Jarvis, guard my room"
- "Avengers assemble"
- "Friday, activate security protocol"

In [1]:

# Cell 2: Install Dependencies
import sys
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    print("📦 Installing required packages for Colab...")
    # !pip install -q SpeechRecognition pyaudio gtts pyttsx3 pygame
else:
    print("💻 Running locally. Ensure requirements.txt is installed.")

# Cell 3: Import Libraries
import speech_recognition as sr
from gtts import gTTS
import pyttsx3
import os
import time
from datetime import datetime
from enum import Enum
import tempfile

class GuardState(Enum):
    """System states for the guard agent"""
    IDLE = "idle"
    LISTENING = "listening"
    ACTIVE = "active"
    SECURED = "secured"

print("✅ Imports successful!")



💻 Running locally. Ensure requirements.txt is installed.
✅ Imports successful!


In [ ]:

# Cell 4: Configuration Class
class AvengersGuardConfig:
    """Configuration for the Avengers Guard System"""
    
    # Activation commands
    ACTIVATION_COMMANDS = [
        "jarvis guard my room",
        "jarvis activate security",
        "avengers assemble",
        "friday activate security protocol",
        "friday guard my room",
        "stark security activate"
    ]
    
    # Deactivation commands
    DEACTIVATION_COMMANDS = [
        "jarvis stand down",
        "avengers stand down",
        "friday deactivate",
        "security off"
    ]
    
    # Recognition settings
    ENERGY_THRESHOLD = 4000  # Adjust based on ambient noise
    PAUSE_THRESHOLD = 0.8
    PHRASE_TIME_LIMIT = 5
    
    # Audio settings
    SAMPLE_RATE = 16000
    
    # Agent personalities (we'll use these in later milestones)
    AGENTS = {
        "iron_man": "Tony Stark's sarcastic but brilliant AI",
        "captain_america": "Steve Rogers' principled protector",
        "black_widow": "Natasha's strategic surveillance"
    }
    
    @staticmethod
    def get_activation_response():
        """Returns a creative Avengers-themed activation response"""
        responses = [
            "Security protocol activated. JARVIS is now monitoring your room.",
            "Avengers Guard System online. Room secured.",
            "FRIDAY here. Perimeter defense active.",
            "Stark Industries Security engaged. All systems operational."
        ]
        import random
        return random.choice(responses)

config = AvengersGuardConfig()
print("⚙️ Configuration loaded!")


⚙️ Configuration loaded!


In [93]:
# Cell 5: Audio Manager with Faster-Whisper
class AudioManager:
    """Manages speech recognition and text-to-speech"""
    
    def __init__(self, use_whisper=True):
        self.recognizer = sr.Recognizer()
        self.recognizer.energy_threshold = 300
        self.recognizer.pause_threshold = 0.8
        self.recognizer.dynamic_energy_threshold = True
        
        # Try to load Faster-Whisper
        self.use_whisper = use_whisper
        if use_whisper:
            try:
                from faster_whisper import WhisperModel
                print("Loading Faster-Whisper model...")
                
                # Use "base" for balance, "tiny" for speed, "small" for accuracy
                self.whisper_model = WhisperModel("small", device="cpu", compute_type="int8")
                print("Faster-Whisper loaded successfully")
            except Exception as e:
                print(f"Whisper not available: {e}")
                print("Using Google Speech API instead")
                self.use_whisper = False
        
        print("Audio Manager initialized")
    
    def listen_for_command(self, timeout=5):
        """Listen for voice command via microphone"""
        try:
            with sr.Microphone() as source:
                print("Listening... (speak now)")                                # In listen_for_command
                self.recognizer.adjust_for_ambient_noise(source, duration=2)  # Longer adjustment
                self.recognizer.dynamic_energy_adjustment_damping = 0.15
                self.recognizer.dynamic_energy_ratio = 1.5
                print(f"   Energy threshold: {self.recognizer.energy_threshold}")
                
                audio = self.recognizer.listen(
                    source, 
                    timeout=timeout,
                    phrase_time_limit=config.PHRASE_TIME_LIMIT
                )
                
            print("Processing speech...")
            
            # Try Faster-Whisper
            if self.use_whisper:
                try:
                    import tempfile
                    import os
                    
                    # Save audio to temp file
                    with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as f:
                        f.write(audio.get_wav_data())
                        temp_path = f.name
                    
                    # Transcribe with Faster-Whisper
                    segments, info = self.whisper_model.transcribe(
                        temp_path,
                        language="en",
                        beam_size=5,
                        vad_filter=True,  # Voice activity detection
                        initial_prompt="Jarvis, Avengers, security, guard, activate, deactivate, stand down"
                    )
                    
                    # Get text from segments
                    text = " ".join([segment.text for segment in segments]).lower().strip()
                    
                    # Cleanup
                    os.unlink(temp_path)
                    
                    if text:
                        print(f"Recognized (Whisper): '{text}'")
                        return True, text
                    else:
                        print("No speech detected")
                        return False, ""
                    
                except Exception as e:
                    print(f"Whisper error: {e}, using Google API")
            
            # Fallback: Google Speech API
            try:
                text = self.recognizer.recognize_google(audio).lower()
                print(f"Recognized (Google): '{text}'")
                return True, text
            except sr.UnknownValueError:
                print("Could not understand audio")
                return False, ""
            except sr.RequestError as e:
                print(f"API error: {e}")
                return False, ""
                    
        except sr.WaitTimeoutError:
            print("Timeout - no speech detected")
            return False, ""
        except Exception as e:
            print(f"Error: {e}")
            return False, ""
    
    def speak(self, text):
        """Convert text to speech using Edge TTS"""
        print(f"Speaking: '{text}'")
        
        try:
            import edge_tts
            import asyncio
            import pygame
            import tempfile
            import os
            
            if not hasattr(self, 'audio_cache'):
                self.audio_cache = {}
            
            # Cache audio files
            if text not in self.audio_cache:
                async def generate_speech():
                    communicate = edge_tts.Communicate(text, "en-US-GuyNeural")
                    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.mp3')
                    temp_file.close()
                    await communicate.save(temp_file.name)
                    return temp_file.name
                
                # FIX: Use nest_asyncio to handle existing event loop
                try:
                    import nest_asyncio
                    nest_asyncio.apply()
                    audio_file = asyncio.run(generate_speech())
                except:
                    # Fallback: create new event loop
                    loop = asyncio.new_event_loop()
                    asyncio.set_event_loop(loop)
                    audio_file = loop.run_until_complete(generate_speech())
                    loop.close()
                
                self.audio_cache[text] = audio_file
            else:
                audio_file = self.audio_cache[text]
            
            # Play audio
            if not pygame.mixer.get_init():
                pygame.mixer.init()
            
            pygame.mixer.music.load(audio_file)
            pygame.mixer.music.play()
            
            while pygame.mixer.music.get_busy():
                pygame.time.Clock().tick(10)
            
            pygame.mixer.music.unload()
            print("Finished speaking")
            
        except Exception as e:
            print(f"TTS Error: {e}")
    
    def cleanup(self):
        """Clean up resources"""
        if hasattr(self, 'audio_cache'):
            for audio_file in self.audio_cache.values():
                try:
                    if os.path.exists(audio_file):
                        os.unlink(audio_file)
                except:
                    pass

print("Audio Manager ready!")

Audio Manager ready!


In [94]:

# Cell 6: Guard State Manager
class GuardStateManager:
    """Manages the state of the guard system"""
    
    def __init__(self):
        self.state = GuardState.IDLE
        self.activation_time = None
        self.audio_manager = AudioManager()
        self.command_history = []
    
    # In Cell 6: GuardStateManager
    def check_activation_command(self, text):
        """Fuzzy match activation commands"""
        from difflib import SequenceMatcher
        
        text = text.lower().strip()
        
        # Check for key phrases
        activation_keywords = ['jarvis', 'avengers', 'friday', 'stark']
        action_keywords = ['guard', 'activate', 'security', 'assemble']
        
        has_trigger = any(keyword in text for keyword in activation_keywords)
        has_action = any(keyword in text for keyword in action_keywords)
        
        if has_trigger and has_action:
            return True
        
        # Fallback: fuzzy match full commands
        for cmd in config.ACTIVATION_COMMANDS:
            similarity = SequenceMatcher(None, text, cmd).ratio()
            if similarity > 0.75:  # 75% match
                return True
        
        return False

    def check_deactivation_command(self, text):
        """Fuzzy match deactivation commands"""
        from difflib import SequenceMatcher
        
        text = text.lower().strip()
        
        # Check for deactivation keywords
        deactivation_keywords = ['stand down', 'deactivate', 'stop', 'off', 'cancel']
        
        if any(keyword in text for keyword in deactivation_keywords):
            return True
        
        # Fallback: fuzzy match
        for cmd in config.DEACTIVATION_COMMANDS:
            similarity = SequenceMatcher(None, text, cmd).ratio()
            if similarity > 0.75:
                return True
        
        return False
        
    def activate(self):
        """Activate guard mode"""
        self.state = GuardState.ACTIVE
        self.activation_time = datetime.now()
        response = config.get_activation_response()
        print(f"\n{'='*60}")
        print(f"🛡️  AVENGERS GUARD ACTIVATED")
        print(f"{'='*60}")
        print(f"⏰ Time: {self.activation_time.strftime('%Y-%m-%d %H:%M:%S')}")
        print(f"🤖 Response: {response}")
        print(f"{'='*60}\n")
        
        self.audio_manager.speak(response)
        return response
    
    def deactivate(self):
        """Deactivate guard mode"""
        self.state = GuardState.IDLE
        duration = (datetime.now() - self.activation_time).total_seconds() if self.activation_time else 0
        response = f"Security protocol deactivated. Room was secured for {int(duration)} seconds."
        print(f"\n{'='*60}")
        print(f"🔓 AVENGERS GUARD DEACTIVATED")
        print(f"{'='*60}")
        print(f"⏱️  Duration: {int(duration)} seconds")
        print(f"{'='*60}\n")
        
        self.audio_manager.speak(response)
        self.activation_time = None
        return response
    
    def log_command(self, command, action):
        """Log command for debugging"""
        entry = {
            'timestamp': datetime.now(),
            'command': command,
            'action': action,
            'state': self.state.value
        }
        self.command_history.append(entry)

print("🎮 State Manager ready!")

🎮 State Manager ready!


In [95]:
# Cell 7: Main Activation Loop
def run_activation_demo(duration=60, stop_on_deactivate=False):
    """
    Run the activation demo for a specified duration
    
    Args:
        duration: How long to run the demo (seconds)
        stop_on_deactivate: If True, exit loop after deactivation
    """
    print(f"\n{'🦾 '*20}")
    print("AVENGERS GUARD ACTIVATION SYSTEM - DEMO MODE")
    print(f"{'🦾 '*20}\n")
    print(f"📋 Instructions:")
    print(f"   - Say activation commands like: 'Jarvis guard my room' or 'Avengers assemble', 'jarvis activate security', 'friday activate security protoco','friday guard my room','stark security activate'")
    print(f"   - Say deactivation commands like: 'Jarvis stand down'")
    print(f"   - Speak clearly and close to your microphone")
    print(f"   - Demo will run for {duration} seconds\n")
    
    # Test microphone first
    print("🔧 Testing microphone...")
    guard = GuardStateManager()
    
    try:
        with sr.Microphone() as source:
            print("✅ Microphone detected")
            guard.audio_manager.recognizer.adjust_for_ambient_noise(source, duration=2)
            print(f"✅ Ambient noise adjusted. Energy threshold: {guard.audio_manager.recognizer.energy_threshold}\n")
    except Exception as e:
        print(f"❌ Microphone error: {e}")
        print("⚠️  Make sure your microphone is connected and not being used by another app\n")
        return None
    
    start_time = time.time()
    
    print("🎤 Starting listening loop...\n")
    
    try:
        while (time.time() - start_time) < duration:
            try:
                # Listen for command with longer timeout
                success, command = guard.audio_manager.listen_for_command(timeout=5)
                
                if success and command:
                    # Check for activation
                    if guard.state == GuardState.IDLE:
                        if guard.check_activation_command(command):
                            guard.activate()
                            guard.log_command(command, "activated")
                        else:
                            print(f"💤 System idle. Command not recognized as activation.\n")
                    
                    # Check for deactivation
                    elif guard.state == GuardState.ACTIVE:
                        if guard.check_deactivation_command(command):
                            guard.deactivate()
                            guard.log_command(command, "deactivated")
                            
                            # NEW: Exit if stop_on_deactivate is True
                            if stop_on_deactivate:
                                print("✅ Exiting demo (deactivated)\n")
                                break
                        else:
                            print(f"🛡️  System active. Command not recognized as deactivation.\n")
                else:
                    print("⏭️  No command detected, continuing...\n")
                
                time.sleep(0.5)
                
            except KeyboardInterrupt:
                print("\n⚠️ Demo interrupted by user")
                break
            except Exception as e:
                print(f"❌ Error: {e}")
                continue
    
    finally:
        # Cleanup
        print("\n🧹 Cleaning up...")
        guard.audio_manager.cleanup()
    
    # Print summary
    print(f"\n{'='*60}")
    print("📊 DEMO SUMMARY")
    print(f"{'='*60}")
    print(f"Total commands logged: {len(guard.command_history)}")
    for entry in guard.command_history:
        print(f"  {entry['timestamp'].strftime('%H:%M:%S')} - {entry['action']}: '{entry['command']}'")
    print(f"{'='*60}\n")
    
    return guard

In [99]:
# Cell 8: Testing Function (Alternative for non-microphone environments)
def test_activation_commands():
    """
    Test activation with pre-defined text commands (no microphone needed)
    Use this in Colab if microphone access is problematic
    """
    print(f"\n{'🧪 '*20}")
    print("TESTING MODE - Text Input Simulation")
    print(f"{'🧪 '*20}\n")
    
    guard = GuardStateManager()
    
    test_commands = [
        ("jarvis guard my room", "should activate"),
        ("hello there", "should ignore"),
        ("avengers assemble", "should activate if idle"),
        ("jarvis stand down", "should deactivate"),
        ("friday activate security protocol", "should activate"),
        ("security off", "should deactivate")
    ]
    
    results = []
    
    for command, expected in test_commands:
        print(f"\n🧪 Testing: '{command}' ({expected})")
        print(f"   Current state: {guard.state.value}")
        
        # Simulate command recognition
        if guard.state == GuardState.IDLE:
            if guard.check_activation_command(command):
                response = guard.activate()  # Returns the response message
                guard.log_command(command, "activated")
                success = True
            else:
                print("   ❌ No activation detected (expected in idle state)")
                success = False
        
        elif guard.state == GuardState.ACTIVE:
            if guard.check_deactivation_command(command):
                response = guard.deactivate()  # Returns the response message
                guard.log_command(command, "deactivated")
                success = True
            elif guard.check_activation_command(command):
                print("   ⚠️ Already active!")
                success = True
            else:
                print("   ❌ No deactivation detected")
                success = False
        
        results.append((command, expected, success))
        time.sleep(1)
    
    # Print test results
    print(f"\n{'='*60}")
    print("📊 TEST RESULTS")
    print(f"{'='*60}")
    success_count = sum(1 for _, _, success in results if success)
    accuracy = (success_count / len(results)) * 100
    
    for cmd, expected, success in results:
        status = "✅" if success else "❌"
        print(f"{status} '{cmd}' - {expected}")
    
    print(f"\n📈 Accuracy: {accuracy:.1f}% ({success_count}/{len(results)})")
    print(f"{'='*60}\n")
    
    return guard, results

print("🧪 Test function ready!")

🧪 Test function ready!


In [100]:

# Cell 9: Run Demo
print("""
🚀 Ready to test Milestone 1!

Choose your testing mode:
1. Run with microphone: run_activation_demo(duration=60)
2. Run text simulation: test_activation_commands()

Example:
>>> guard = test_activation_commands()  # Safe for Colab
>>> # OR
>>> guard = run_activation_demo(30)  # Requires microphone
""")

# Cell 10: Milestone 1 Validation
def validate_milestone_1():
    """
    Validates that Milestone 1 requirements are met:
    - Command recognition (90%+ accuracy target)
    - State management (guard mode on/off)
    - Audio feedback
    """
    print("🔍 VALIDATING MILESTONE 1 REQUIREMENTS\n")
    
    checklist = {
        "✅ Speech recognition (ASR) implemented": True,
        "✅ Activation command detection": True,
        "✅ State management (idle/active)": True,
        "✅ Audio feedback (TTS)": True,
        "✅ Command logging": True,
        "✅ Error handling": True
    }
    
    for item, status in checklist.items():
        print(f"{item}")
    
    print(f"\n{'='*60}")
    print("🎯 MILESTONE 1 STATUS: COMPLETE ✅")
    print(f"{'='*60}")
    print("\n📝 Next Steps:")
    print("   → Test with your microphone")
    print("   → Record demo video (30 seconds)")
    print("   → Move to Milestone 2: Face Recognition")
    print(f"{'='*60}\n")

validate_milestone_1()


🚀 Ready to test Milestone 1!

Choose your testing mode:
1. Run with microphone: run_activation_demo(duration=60)
2. Run text simulation: test_activation_commands()

Example:
>>> guard = test_activation_commands()  # Safe for Colab
>>> # OR
>>> guard = run_activation_demo(30)  # Requires microphone

🔍 VALIDATING MILESTONE 1 REQUIREMENTS

✅ Speech recognition (ASR) implemented
✅ Activation command detection
✅ State management (idle/active)
✅ Audio feedback (TTS)
✅ Command logging
✅ Error handling

🎯 MILESTONE 1 STATUS: COMPLETE ✅

📝 Next Steps:
   → Test with your microphone
   → Record demo video (30 seconds)
   → Move to Milestone 2: Face Recognition



In [98]:
# Text simulation (no mic needed)
#guard, results = test_activation_commands()

# Live with microphone
guard = run_activation_demo(duration=60, stop_on_deactivate=True)


🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 
AVENGERS GUARD ACTIVATION SYSTEM - DEMO MODE
🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 🦾 

📋 Instructions:
   - Say activation commands like: 'Jarvis guard my room' or 'Avengers assemble', 'jarvis activate security', 'friday activate security protoco','friday guard my room','stark security activate'
   - Say deactivation commands like: 'Jarvis stand down'
   - Speak clearly and close to your microphone
   - Demo will run for 60 seconds

🔧 Testing microphone...
Loading Faster-Whisper model...
Faster-Whisper loaded successfully
Audio Manager initialized
✅ Microphone detected
✅ Ambient noise adjusted. Energy threshold: 6.863922938945922

🎤 Starting listening loop...

Listening... (speak now)
   Energy threshold: 329.9094302784575
Timeout - no speech detected
⏭️  No command detected, continuing...

Listening... (speak now)
   Energy threshold: 6.186081089066687
Processing speech...
No speech detected
⏭️  No command detected, continuing...

Listening...

In [101]:
guard, results = test_activation_commands()


🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 
TESTING MODE - Text Input Simulation
🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 🧪 

Loading Faster-Whisper model...
Faster-Whisper loaded successfully
Audio Manager initialized

🧪 Testing: 'jarvis guard my room' (should activate)
   Current state: idle

🛡️  AVENGERS GUARD ACTIVATED
⏰ Time: 2025-10-04 22:02:27
🤖 Response: FRIDAY here. Perimeter defense active.

Speaking: 'FRIDAY here. Perimeter defense active.'
Finished speaking

🧪 Testing: 'hello there' (should ignore)
   Current state: active
   ❌ No deactivation detected

🧪 Testing: 'avengers assemble' (should activate if idle)
   Current state: active
   ⚠️ Already active!

🧪 Testing: 'jarvis stand down' (should deactivate)
   Current state: active

🔓 AVENGERS GUARD DEACTIVATED
⏱️  Duration: 9 seconds

Speaking: 'Security protocol deactivated. Room was secured for 9 seconds.'
Finished speaking

🧪 Testing: 'friday activate security protocol' (should activate)
   Current state: idle

🛡️  AVENGERS G